In [1]:
from qbraid import device_wrapper, get_devices
from qbraid.devices._utils import SUPPORTED_VENDORS

In [2]:
import cirq
import numpy as np
from braket.circuits import Circuit as BraketCircuit
from qiskit import QuantumCircuit as QiskitCircuit
import networkx as nx
import dwave_networkx as dnx

In [4]:
dwave_wrapper = device_wrapper("DW_2000Q_6", "D-Wave", vendor="AWS")
dwave_sampler = dwave_wrapper.get_sampler()

In [5]:
n = 5
m = 10
graph = nx.gnm_random_graph(n, m, seed=42)

In [6]:
result = dnx.min_vertex_cover(graph, dwave_sampler, answer_mode="histogram")
print('Result to MVC problem:', result)
print('Size of the vertex cover:', len(result))

Result to MVC problem: [0, 1, 2, 4]
Size of the vertex cover: 4


In [7]:
def device_wrapper_input(vendor):
    input_list = []
    for provider in SUPPORTED_VENDORS[vendor]:
        for device in SUPPORTED_VENDORS[vendor][provider]:
            if provider != "D-Wave":
                data = (device, provider, vendor)
                input_list.append(data)
    return input_list

In [8]:
def braket_circuit():
    circuit = BraketCircuit()
    circuit.h(0)
    circuit.ry(0, np.pi / 2)
    return circuit

In [9]:
def cirq_circuit(meas=True):
    q0 = cirq.GridQubit(0, 0)
    def test_circuit():
        yield cirq.H(q0)
        yield cirq.Ry(rads=np.pi / 2)(q0)
        if meas:
            yield cirq.measure(q0, key='q0')
    circuit = cirq.Circuit()
    circuit.append(test_circuit())
    return circuit

In [10]:
def qiskit_circuit(meas=True):
    circuit = QiskitCircuit(1, 1) if meas else QiskitCircuit(1)
    circuit.h(0)
    circuit.ry(np.pi / 2, 0)
    if meas:
        circuit.measure(0, 0)
    return circuit